## Практическое задание

1. Обучите любую модель классификации на датасете IRIS до применения PCA и после него. Сравните качество классификации по отложенной выборке.
2. Напишите свою реализацию метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [2]:
# Для начала масштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование — вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [3]:
# Найдем собственные векторы и собственные значения (англ. Eigenvalues)
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(437.77467247979905, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021032, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569723, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292853, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [4]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329992, 22.850761786701742, 3.668921889282875, 0.517870910715476]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [5]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную, то есть накапливаемую, дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329992, 22.850761786701742, 3.668921889282875, 0.517870910715476]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [6]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [7]:
# Сформируем новую матрицу «объекты-признаки»
Z = X_.dot(W)

In [8]:
X_train_before, X_test_before, y_train_before, y_test_before = train_test_split(X_, Y, test_size=0.3)
X_train_after, X_test_after, y_train_after, y_test_after = train_test_split(Z, Y, test_size=0.3)

rf = RandomForestClassifier(max_depth=10, n_estimators=20)
rf.fit(X_train_before, y_train_before)
y_pred_before = rf.predict(X_test_before)

rf = RandomForestClassifier(max_depth=10, n_estimators=20)
rf.fit(X_train_after, y_train_after)
y_pred_after = rf.predict(X_test_after)

print(accuracy_score(y_test_after, y_pred_after))
print(accuracy_score(y_test_before, y_pred_before))

0.8444444444444444
0.9555555555555556


**Реализация метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().**

In [9]:
def get_svd_pca(X, n):
    C = np.dot(X.T, X) / (n-1)
    eigen_vals, eigen_vecs = np.linalg.eig(C)
    U, Sigma, Vh = np.linalg.svd(X, 
        full_matrices=False, 
        compute_uv=True)
    return np.allclose(np.square(Sigma) / (n - 1), eigen_vals)